Reference: 
https://colab.research.google.com/github/styler00dollar/Colab-mmediting/blob/master/Colab-mmediting.ipynb?authuser=1#scrollTo=h75uv5KNrNBZ

In [ ]:
!python -V

Python 3.7.13


In [ ]:
import torch
torch.__version__

'1.10.0+cu111'

In [ ]:
%cd /content/


# !pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu102/torch1.9.0/index.html
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

!git clone https://github.com/open-mmlab/mmediting mmediting

%cd mmediting
!pip install -r requirements.txt
!pip install -v -e .  # or "python setup.py develop"
!pip install tensorboard

In [ ]:
%cd /content/
!mkdir /content/models/
# deepfillv2
!wget --no-check-certificate 'https://openmmlab.oss-accelerate.aliyuncs.com/mmediting/inpainting/deepfillv2/deepfillv2_256x256_8x2_places_20200619-10d15793.pth' -O /content/models/deepfillv2_256x256_8x2_places.pth

!wget --no-check-certificate 'https://openmmlab.oss-accelerate.aliyuncs.com/mmediting/inpainting/deepfillv2/deepfillv2_256x256_8x2_celeba_20200619-c96e5f12.pth' -O /content/models/deepfillv2_256x256_8x2_celeba.pth
%cd /content/mmediting



In [ ]:
#@title Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Google Drive connected.')

Mounted at /content/drive
Google Drive connected.


In [ ]:
#no need to run this
"""

import os
cwd = os.path.join(r'/content/drive/MyDrive/MTECH PROJECT/root','data','hateful_memes')
maskPath =  os.path.join(cwd,'img_mask_3px')
inpaintResultPath =  os.path.join(cwd,'img_clean')
print(maskPath,inpaintResultPath)
"""

"\n\nimport os\ncwd = os.path.join(r'/content/drive/MyDrive/MTECH PROJECT/root','data','hateful_memes')\nmaskPath =  os.path.join(cwd,'img_mask_3px')\ninpaintResultPath =  os.path.join(cwd,'img_clean')\nprint(maskPath,inpaintResultPath)\n"

In [ ]:
!ls /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/img/ | wc -l
!ls /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/masks/ | wc -l
!ls /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/textless_img/ | wc -l
# !ls /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/img_mask_3px/ | wc -l

In [ ]:
# !mkdir /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/textless_img
!cp -n /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/img_mask_3px/*.mask.png /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/textless_img/

cp: cannot stat '/content/drive/MyDrive/MTECH PROJECT/root/data/hateful_memes/img_mask_3px/*.mask.png': No such file or directory


In [ ]:
# !mkdir /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/masks
# !ls /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/img_mask_3px/[0-9][0-9][0-9][0-9][0-9].png | wc -l

!cp -n /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/img_mask_3px/[0-9][0-9][0-9][0-9][0-9].png /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/masks/

cp: cannot stat '/content/drive/MyDrive/MTECH PROJECT/root/data/hateful_memes/img_mask_3px/[0-9][0-9][0-9][0-9][0-9].png': No such file or directory


In [ ]:
!ls /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/img_clean

ls: cannot access '/content/drive/MyDrive/MTECH PROJECT/root/data/hateful_memes/img_clean': No such file or directory


Batch Prediction

In [ ]:
%%writefile /content/mmediting/demo/batch_inpaint.py

import argparse

import mmcv
import torch

from mmedit.apis import init_model, inpainting_inference
from mmedit.core import tensor2img

import glob
from tqdm import tqdm
import os

def parse_args():
    parser = argparse.ArgumentParser(description='Inpainting demo')
    parser.add_argument('--config', help='test config file path')
    parser.add_argument('--checkpoint', help='checkpoint file')
    parser.add_argument('--input', help='input images')
    parser.add_argument('--masks', help='monochrome masks')
    parser.add_argument('--output', help='path to save inpainting result')
    parser.add_argument(
        '--imshow', action='store_true', help='whether show image with opencv')
    parser.add_argument('--device', type=int, default=0, help='CUDA device id')
    args = parser.parse_args()
    return args



def main():
    args = parse_args()

    model = init_model(
        args.config, args.checkpoint, device=torch.device('cuda', args.device))

    # original image
    files = glob.glob(args.input + '/*.png', recursive=True)
    files_jpg = glob.glob(args.input + '/*.jpg', recursive=True)
    files.extend(files_jpg)


    for f in tqdm(files):

      # creating mask

      #image_filepath = f
      print(f)
      mask_filepath = os.path.join(args.masks, os.path.splitext(os.path.basename(f))[0] + '.mask.png')      
      save_path = os.path.join(args.output, os.path.splitext(os.path.basename(f))[0] + '.png')      
      result = inpainting_inference(model, f, mask_filepath)
      result = tensor2img(result, min_max=(-1, 1))[..., ::-1]

      

      mmcv.imwrite(result, save_path)

      #if args.imshow:
      #    mmcv.imshow(result, 'predicted inpainting result')


if __name__ == '__main__':
    main()

Writing /content/mmediting/demo/batch_inpaint.py


In [ ]:
!python demo/batch_inpaint.py \
    --config configs/inpainting/deepfillv2/deepfillv2_256x256_8x2_places.py \
    --checkpoint /content/models/deepfillv2_256x256_8x2_places.pth \
    --input /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/textless_img/ \
    --masks /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/masks/ \
    --output /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/img_clean

Traceback (most recent call last):
  File "demo/batch_inpaint.py", line 7, in <module>
    from mmedit.apis import init_model, inpainting_inference
  File "/content/mmediting/mmedit/apis/__init__.py", line 4, in <module>
    from .matting_inference import init_model, matting_inference
  File "/content/mmediting/mmedit/apis/matting_inference.py", line 8, in <module>
    from mmedit.models import build_model
  File "/content/mmediting/mmedit/models/__init__.py", line 2, in <module>
    from .backbones import *  # noqa: F401, F403
  File "/content/mmediting/mmedit/models/backbones/__init__.py", line 14, in <module>
    from .sr_backbones import (EDSR, LIIFEDSR, LIIFRDN, RDN, SRCNN, BasicVSRNet,
  File "/content/mmediting/mmedit/models/backbones/sr_backbones/__init__.py", line 3, in <module>
    from .basicvsr_pp import BasicVSRPlusPlus
  File "/content/mmediting/mmedit/models/backbones/sr_backbones/basicvsr_pp.py", line 6, in <module>
    from mmcv.ops import ModulatedDeformConv2d, modula

In [ ]:
!ls /content/drive/MyDrive/MTECH\ PROJECT/root/data/hateful_memes/img_clean/ | wc -l

ls: cannot access '/content/drive/MyDrive/MTECH PROJECT/root/data/hateful_memes/img_clean/': No such file or directory
0


RUN OID.iypnb after this